In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [2]:
def url_add_http(url):
    """ 
    Add https:// on url if it does not contain one
    """
    string_r = r"^(https://)"
    string_e = re.compile(string_r)
    if not string_e.search(url):
        url = f"https://{url}"
    return url

In [4]:
def url_test(url):
    """ Allows to test a url and to know if url has a status code of 200 """
    
    # msg of redirection
    redi_text = "If you are not redirected automatically, follow the www.ioam.de"
    
    headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0"}

    # increase of max_redirect
    #session = requests.Session()
    #session.max_redirects = 100000000000
    try:
        url = url_add_http(url)
        response = requests.get(url, timeout=60, headers= headers)
    except Exception:
        # if max_redirects has been exceeded
        response = requests.get(url, timeout=60, headers= headers, allow_redirects=False)
    
    # if the page content contains a redirection message to another site    
    soup = BeautifulSoup(response.text, "html.parser")
    tags = soup.find_all("body")
    header = soup.find_all("head")
    content = [" ".join(tag.stripped_strings) for tag in tags]
    content = " ".join(content).split(" ")[:-1]
    redi_text = redi_text.split(" ")[:-1]

    return set(content) != set(redi_text) and response.status_code == 200 and header != []

In [3]:
def reformat_url(url):
    """ reformat the domain name to get base url"""
    url_split = url.split(".")
    return f"{url_split[-2]}.{url_split[-1]}"

In [4]:
def get_url(url):
    """ get url with which the page content will be parsed """
    if not url_test(url):
        url = reformat_url(url)
    url = url_add_http(url)
    return url

In [1]:
import requests
import concurrent.futures # pour exécuter plusieurs tâches en parallèle
from tqdm import tqdm

def get_valid_domains(file_name, new_file_name):
    '''Définition de la fonction pour extraire les noms de domaine valides'''
    
    valid_domains = set() # pour stocker les noms de domaine valides
    with open(file_name) as f:
        for line in f:
            line = line.strip() # Supprimer les espaces blancs en début et fin de ligne
            if line.startswith('www.') or line.count('.') == 1:
                domain = f"https://{line}" # Ajouter le préfixe 'https://
                valid_domains.add(domain)

    total_domains = len(valid_domains)
    with open(new_file_name, 'w') as f:
        # Créer un pool d'exécution pour exécuter les tâches en parallèle
        with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
            # Soumettre chaque nom de domaine à la fonction test_domain pour vérifier s'il est valide
            futures = [executor.submit(test_domain, domain) for domain in valid_domains]
            with tqdm(total=total_domains) as pbar: # Créer une barre de progression 
                for future in concurrent.futures.as_completed(futures): 
                    if valid_domain := future.result(): # Écrire les noms de domaine valides dans un nouveau fichier
                        f.write(valid_domain + '\n')
                    pbar.update() # Mettre à jour la barre de progression pour refléter l'avancement
                    
def test_domain(domain):  
    '''Définition de la fonction pour tester la validité du nom de domaine'''
    
    if domain.startswith('https://www.'):
        return domain
    try:
        with requests.head(domain, timeout=5) as response: # faire une requête HTTP HEAD 
            if response.status_code == 200:
                return domain
    except Exception:
        pass

if __name__ == '__main__':
    get_valid_domains('news.txt', 'urls_news.txt')


100%|██████████| 2531/2531 [04:18<00:00,  9.78it/s]


In [ ]:
# unique
import re
import requests

def get_valid_domains(file_name, new_file_name):
    unique_domains = set()
    for line in open(file_name):
        # Supprimer les caractères spéciaux en début et fin de ligne
        line = line.strip()

        # Vérifier si le nom de domaine commence par "www" ou a une structure de la forme "mot.mot"
        if line.startswith('www.') or re.match(r'^\w+\.\w+$', line):
            # Ajouter "https://" devant chaque nom de domaine sélectionné
            domain = f"https://{line}"

            # Effectuer une requête HTTP sur chaque nom de domaine sélectionné
            try:
                response = requests.get(domain)
            except Exception:
                continue

            # Vérifier si le statut de réponse HTTP est 200
            if (
                response.status_code == 200
                and response.content not in unique_domains
            ):
                unique_domains.add(response.content)
                # Écrire les noms de domaine valides dans un nouveau fichier
                with open(new_file_name, 'a') as f:
                    f.write(domain + '\n')


if __name__ == '__main__':
    get_valid_domains('news.txt', 'urls_news.txt')
